In [ ]:
#Importing TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
import numpy as np

tf.__version__

In [ ]:
# Importing and Authenticating GEE Python API
!pip install -U earthengine-api --no-deps --quiet
import ee
from google.colab import auth
auth.authenticate_user()
ee.Authenticate()
ee.Initialize()

In [ ]:
#GCS Configuration
PROJECT = 'your_google_cloud_console_project_name'
REGION = 'your_google_cloud_storage_bucket_region'

# Some Information about training data
FEATURE_NAMES = ['B4','B3','B2','NDVI','LC']
BANDS = ['B4','B3','B2','NDVI']
LABEL = 'LC'
N_CLASSES = 4 #Number of Classes

# Imports (Training and Testing Data)
TRAIN_FILE_PATH = 'training_data_path'
TEST_FILE_PATH = 'testing_data_path'

# (About Model that will later be exported)
MODEL_DIR = 'gcs_bucket_path/Model_Folder_Name'
MODEL_NAME = 'Model_Name'
VERSION_NAME = 'V0' #Optional

# Exported Image Detaiks
EXPORTED_IMAGE_PREFIX = 'Image_Prefix_Name' #exported image from GEE JS API to GCS **Important**
OUTPUT_IMAGE_FILE = 'gcs_bucket_dir/fileName.TFRecord'
OUTPUT_ASSET_ID = 'projects/your_GEE_JS_API_Assets_project_name/assets/imageName'

In [ ]:
# Create a dataset from the TFRecord file in Cloud Storage. (Prepare for TensorFlow)
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# Print the first record to check.
iter(train_dataset).next()

In [ ]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]


# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
iter(parsed_dataset).next()


In [ ]:
# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Map the to_tuple function
input_dataset = parsed_dataset.map(to_tuple)

print(input_dataset,'Printed \n \n')
iter(input_dataset).next()

# Our Deep Neural Network Model Defind Here

In [ ]:
# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

In [ ]:
# Compile the model with the specified loss function.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
input_dataset = input_dataset.shuffle(64).batch(8)
iter(input_dataset).next() #printing the first training record just for check

In [ ]:
# Fitting the model
model.fit(x=input_dataset, epochs=100)

In [ ]:
# Model Summary and Saving the model
model.summary()
model.save(MODEL_DIR, save_format='tf')

# Accuracy Assessment

In [ ]:
# CHECK THE MODEL accuracy on the validation dataset
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .batch(2))

model.evaluate(test_dataset)

# Loading the image dataset patches

In [ ]:
# Classifying The Image
# We will now use the DNN model to predict the output class for all pixels of the input image.
# The input composite for the entire basin has been exported to GCS. We will read the image and run prediction for each image path.
# When you export an image from Earth Engine as TFRecords, you get 2 files
# .tfrecord.gz files containing image patches
# mixer.json file containing image metadata and georeferencing information

# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'path_to_gcs_bucket'

# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if EXPORTED_IMAGE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

print(image_files_list)

In [ ]:
# Make sure the files (patches) are in the right order.
image_files_list.sort()
image_files_list


In [ ]:
import json
# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
mixer

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

In [ ]:
# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)
iter(image_dataset).next() #Display first record from batch

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

In [ ]:
# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])
print(predictions[0].shape)

In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[]]
cur_patch = 1
for prediction in predictions:
  patch[0].append(int(tf.argmax(prediction, 1)))

  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(value=patch[0]))
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[]]
    cur_patch += 1

writer.close()

In [ ]:
#Upload back to GEE
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

My Special Thanks to Dr. Ujaval Gandhi (Spatial Thoughts) and Dr. Hammad Gilani (IWMI - Pakistan) for helping me learning the process.

Watch the webinar here: https://www.youtube.com/watch?v=tCrM3Lm-AM4

YouTube Link to my YouTube Channel: https://youtube.com/muddasirshah

Email me at: muddasirshah@outlook.com

Dr. Ujaval Gandhi:
https://satialthoughts.com
https://www.youtube.com/SpatialThoughts

Some Useful Resources:

https://developers.google.com/earth-engine/guides/tf_examples

https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.14874&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false
